# OpenStreetMap

In [ ]:
import requests
import pandas as pd
import overpy

In [ ]:
api = overpy.Overpass()

In [ ]:
r = api.query("""
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
""")

In [ ]:
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [ ]:
coords

In [ ]:
cat_list = '4bf58dd8d48988d10d941735,4bf58dd8d48988d116941735,50327c8591d4c4b30a586d5d'

In [ ]:
names = ['biergarten']
lat = [48.1376089]
lon = [11.5803564]

# 48.1376089 / 11.5803564

In [ ]:
munich_venues = getNearbyVenues(CLIENT_ID, CLIENT_SECRET, VERSION, names, lat, lon, cat_list, radius=25)

In [ ]:
munich_venues

In [ ]:
response = requests.get('https://nominatim.openstreetmap.org/search?country=Germany&city=Frankfurt&format=json').json()

In [ ]:
response[0]

In [ ]:
response[0]['osm_id'] + 3600000000

In [ ]:
city_code = str(response[0]['osm_id'] + 3600000000)

In [ ]:
def get_city_id(city_name):
    
    response = requests.get('https://nominatim.openstreetmap.org/search?country=Germany&city=' + city_name + '&format=json').json()
    
    return str(response[0]['osm_id'] + 3600000000)

In [ ]:
r = api.query(
"[out:json];"
"area(" + city_code + ")->.searchArea;"
"("
  "node[amenity=biergarten](area.searchArea);"
  "way[amenity=biergarten](area.searchArea);"
  "relation[amenity=biergarten](area.searchArea);"
");"
"(._;>;);"
"out center;"
)

In [ ]:
def overpass_query(city_code):

    r = api.query(
        "[out:json];"
        "area(" + city_code + ")->.searchArea;"
        "("
          "node[amenity=biergarten](area.searchArea);"
          "way[amenity=biergarten](area.searchArea);"
          "relation[amenity=biergarten](area.searchArea);"
        ");"
        "(._;>;);"
        "out center;"
        )
        
    return r

In [ ]:
len(r.ways)

In [ ]:
len(r.nodes)

In [ ]:
len(r.relations)

In [ ]:
lat = float(r.ways[0].center_lat)
lon = float(r.ways[0].center_lon)

In [ ]:
r.nodes[14].id

In [ ]:
r.nodes[14].tags['amenity']=='biergarten'

In [ ]:
not r.nodes[13].tags

In [ ]:
# Nodes
for i, node in enumerate(r.nodes):
    print('latitude: {}'.format(float(r.nodes[i].lat)))
    print('longitude: {}'.format(float(r.nodes[i].lon)))

In [ ]:
# Ways
for i, way in enumerate(r.ways):
    print('latitude: {}'.format(float(r.ways[i].center_lat)))
    print('longitude: {}'.format(float(r.ways[i].center_lon)))

In [ ]:
r.nodes[0].id

In [ ]:
tmp_df = pd.DataFrame()
d = {
    'id': r.nodes[0].id,
    'type':'Node',
    'latitude':float(r.nodes[0].lat),
    'longitude':float(r.nodes[0].lon)
}
tmp_df= tmp_df.append(d, ignore_index=True)
tmp_df

In [ ]:
'amenity' in r.nodes[14].tags.keys()

In [ ]:
def parse_nodes(response):
    
    res_df = pd.DataFrame()
    
    # Nodes
    for i, node in enumerate(response.nodes):
        
        if not not response.nodes[i].tags:
            
            if 'amenity' in response.nodes[i].tags.keys():
            
                if response.nodes[i].tags['amenity']=='biergarten':

                    d = {
                    'id': response.nodes[i].id,
                    'type':'Node',
                    'latitude':float(response.nodes[i].lat),
                    'longitude':float(response.nodes[i].lon),
                    'tags':response.nodes[i].tags
                    }
                    res_df = res_df.append(d, ignore_index=True)

    return res_df

In [ ]:
df = parse_nodes(r)

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
r.ways[0].tags

In [ ]:
def parse_ways(response):
    
    res_df = pd.DataFrame()
    
    # Nodes
    for i, way in enumerate(response.ways):
        
        if not not response.ways[i].tags:

            if 'amenity' in response.ways[i].tags.keys():

                if response.ways[i].tags['amenity']=='biergarten':
                    
                    d = {
                    'id': response.ways[i].id,
                    'type':'Way',
                    'latitude':float(response.ways[i].center_lat),
                    'longitude':float(response.ways[i].center_lon),
                    'tags':response.ways[i].tags
                    }
                    res_df = res_df.append(d, ignore_index=True)
        
    return res_df

In [ ]:
df = parse_ways(r)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
r.relations[0].tags

In [ ]:
def parse_relations(response):
    
    res_df = pd.DataFrame()
    
    # Nodes
    for i, relation in enumerate(response.relations, start=0):
        
        if not not response.relations[i].tags:

            if 'amenity' in response.relations[i].tags.keys():

                if response.relations[i].tags['amenity']=='biergarten':
                    
                    d = {
                    'id': response.relations[i].id,
                    'type':'Relation',
                    'latitude':float(response.relations[i].center_lat),
                    'longitude':float(response.relations[i].center_lon),
                    'tags':response.relations[i].tags
                    }
                    res_df = res_df.append(d, ignore_index=True)
        
    return res_df

In [ ]:
df = parse_relations(r)
print(df.shape)
print(type(df))
df.head()

In [ ]:
def parse_response(response):
    
    nodes_df = parse_nodes(response)
    ways_df = parse_ways(response)
    relations_df = parse_relations(response)
    
    response_df = pd.DataFrame()
    if not nodes_df.empty:
        response_df = response_df.append(nodes_df)
    if not ways_df.empty:
        response_df = response_df.append(ways_df)
    if not relations_df.empty:
        response_df = response_df.append(relations_df)
    
    return response_df.reset_index(drop=True)  
    

In [ ]:
res_df = parse_response(r)
print(res_df.shape)
res_df.tail()

In [ ]:
def get_biergartens_for_city(city_name):
    
    city_id = get_city_id(city_name)
    
    response = overpass_query(city_id)
    
    res_df = parse_response(response)
    
    res_df['city'] = city_name
    res_df['city_id'] = city_id
    
    return res_df

In [ ]:
city = 'Hamburg'
df = get_biergartens_for_city(city)
print(df.shape)
df.head()

In [ ]:
df.tail()

# Testing modules

In [1]:
from openstreetmap import openstreetmap

In [2]:
op = openstreetmap.OpenStreetMap('Munich')

In [3]:
df = op.get_biergartens_for_city()

In [4]:
df.shape

(131, 7)

In [5]:
df.head()

,id,latitude,longitude,tags,type,city,city_id
0,118252923.0,48.149801,11.585537,"{'addr:city': 'München', 'addr:housenumber': '...",Node,Munich,3600062428
1,260796705.0,48.120427,11.607181,"{'addr:city': 'München', 'addr:country': 'DE',...",Node,Munich,3600062428
2,263389412.0,48.114720,11.614455,"{'amenity': 'biergarten', 'name': 'Alter Wirt'...",Node,Munich,3600062428
3,263904864.0,48.114070,11.613793,"{'amenity': 'biergarten', 'name': 'Zar', 'open...",Node,Munich,3600062428
4,264012141.0,48.130373,11.615910,"{'amenity': 'biergarten', 'name': 'Padova', 'w...",Node,Munich,3600062428


In [6]:
df.tail()

,id,latitude,longitude,tags,type,city,city_id
126,607507205.0,48.128821,11.653664,"{'addr:city': 'München', 'addr:housenumber': '...",Way,Munich,3600062428
127,624027883.0,48.138347,11.562754,"{'addr:housenumber': '10', 'addr:street': 'Ado...",Way,Munich,3600062428
128,701098863.0,48.163090,11.532503,"{'amenity': 'biergarten', 'name': 'Taxisgarten...",Way,Munich,3600062428
129,705288640.0,48.132953,11.588964,"{'addr:city': 'München', 'addr:country': 'DE',...",Way,Munich,3600062428
130,774759219.0,48.145528,11.593558,"{'amenity': 'biergarten', 'name': 'Leib und Se...",Way,Munich,3600062428


In [7]:
op.no_of_biergartens

131

# Foursquare

## Venue Categories
https://developer.foursquare.com/docs/build-with-foursquare/categories/

* German Restaurant 4bf58dd8d48988d10d941735
    * Bavarian Restaurant 56aa371ce4b08b9a8d573572
* Bar 4bf58dd8d48988d116941735
    * Beer Bar 56aa371ce4b08b9a8d57356c
    * Beer Garden 4bf58dd8d48988d117941735
    * Pub 4bf58dd8d48988d11b941735
* Brewery 50327c8591d4c4b30a586d5d


### Define Foursquare Credentials and Version

In [33]:
# Import password file for foursquare
import os
import sys
sys.path.insert(0, os.path.abspath(os.environ['USERPROFILE'] + '/Desktop/my_pwd'))
import foursquare_pwd

In [34]:
# Map the imported variable to the ones in notebook
CLIENT_ID = foursquare_pwd.client_id # your Foursquare ID
CLIENT_SECRET = foursquare_pwd.client_secret # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# Determine the max number of results
LIMIT = 100

In [ ]:
names = ['Munich']
latitudes = [48.1351]
longitudes = [11.5820]

In [ ]:
def getNearbyVenues(fsqr_client_id, fsqr_client_secret, version, names, latitudes, longitudes, limit = 100, radius=1000):
    
    venues_list=[]
    for lat, lng, name in zip(latitudes, longitudes, names):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            fsqr_client_id, 
            fsqr_client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Name Latitude', 
                  'Name Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

In [ ]:
def get_nearby_venues(fsqr_client_id, fsqr_client_secret, version, names, latitudes, longitudes, radius=15, limit = 100):
    
    venues_list=[]
    for lat, lng, name in zip(latitudes, longitudes, names):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{} \
        &categoryId=4bf58dd8d48988d10d941735,4bf58dd8d48988d116941735,50327c8591d4c4b30a586d5d&radius={}&limit={}&intent=browse'.format(
            fsqr_client_id, 
            fsqr_client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'], 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Name Latitude', 
                  'Name Longitude', 
                  'venue_id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

* German Restaurant 4bf58dd8d48988d10d941735
    * Bavarian Restaurant 56aa371ce4b08b9a8d573572
* Bar 4bf58dd8d48988d116941735
    * Beer Bar 56aa371ce4b08b9a8d57356c
    * Beer Garden 4bf58dd8d48988d117941735
    * Pub 4bf58dd8d48988d11b941735
* Brewery 50327c8591d4c4b30a586d5d

In [ ]:
cat_list = '4bf58dd8d48988d10d941735,4bf58dd8d48988d116941735,50327c8591d4c4b30a586d5d'

In [ ]:
venues = getNearbyVenues(CLIENT_ID, CLIENT_SECRET, VERSION, df['city'], df['latitude'], df['longitude'], limit=500)

In [ ]:
pd.set_option('display.max_rows', 200)

In [ ]:
venues

In [ ]:
venues.shape

In [ ]:
def getNearbyVenues2(fsqr_client_id, fsqr_client_secret, version, names, latitudes, longitudes, radius=15, limit = 100):
    
    venues_list=[]
    for lat, lng, name in zip(latitudes, longitudes, names):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{} \
        &categoryId=4bf58dd8d48988d10d941735,4bf58dd8d48988d116941735,50327c8591d4c4b30a586d5d&radius={}&limit={}&intent=browse'.format(
            fsqr_client_id, 
            fsqr_client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        response = requests.get(url).json()
    return response

In [ ]:
df2 = pd.DataFrame(df.tail(1))
df2

In [ ]:
response = getNearbyVenues2(CLIENT_ID, CLIENT_SECRET, VERSION, df2['city'], df2['latitude'], df2['longitude'], limit=500)

In [ ]:
response

# Test modules

In [10]:
from foursquare import foursquare

In [11]:
v = foursquare.Venues(CLIENT_ID, CLIENT_SECRET)

In [18]:
v.radius = 25

In [19]:
names = df['city']
latitudes = df['latitude']
longitudes = df['longitude']
v.get_nearby_venues(names, latitudes, longitudes)

KeyError: 'groups'

In [ ]:
v.venue_df

# Reviews

In [39]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=1T5BRNX20QABCKZSPYCIDY3UZACVDTDZOF5NYOLOWXGTSBDR&client_secret=MBNK2BJMPMQTAKEJJ21OO42DTLIFY0H2P5WLSE4WWCTVBJBV&v=20180605'

In [22]:
import requests

In [40]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': '?', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'name': '

In [41]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.0


In [42]:
result['response']['venue']['likes']['count']

120

In [30]:
import pandas as pd

In [50]:
def get_venue_review(venue_id):
    
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    response = requests.get(url).json()
    
    res_df = pd.DataFrame(columns=['venue_id','venue_name','rating','likes_cnt'])        
    d = {
    'venue_id': venue_id,
    'venue_name': response['response']['venue']['name'],
    'rating': response['response']['venue']['rating'],
    'likes_cnt': response['response']['venue']['likes']['count']
    }
    res_df = res_df.append(d, ignore_index=True)
    
    return res_df

In [47]:
result['response']['venue']['likes']['count']

120

In [51]:
get_venue_review(venue_id)

,venue_id,venue_name,rating,likes_cnt
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,7.0,120
